In [ ]:
# The Google Earth Engine module
import ee

# The datetime module is used to specify the dates
# to search for imagery
import datetime

# Import the geemap (https://geemap.org/) module which
# has a visualisation tool
import geemap

# Geopandas allows us to read the shapefile used to
# define the region of interest (ROI)
import geopandas

# The colab module to access data from your google drive
from google.colab import drive

In [ ]:
try:
  import pb_gee_tools
  import pb_gee_tools.datasets
except:
  !git clone https://github.com/remotesensinginfo/pb_gee_tools.git
  !pip install ./pb_gee_tools/.
  import pb_gee_tools
  import pb_gee_tools.datasets

In [ ]:
ee_prj_name = "ee-pb-dev"  # <==== Replace this with your own EE project string
ee.Authenticate()
ee.Initialize(project=ee_prj_name)

In [ ]:
drive.mount("/content/drive")

In [ ]:
# The region of interest
vec_roi_file = "/content/drive/MyDrive/burnt_veg/burnt_area_roi.geojson"

# Dates before the fire
pre_burn_start_date = datetime.datetime(year=2015, month=5, day=1)
pre_burn_end_date = datetime.datetime(year=2015, month=7, day=31)

# Dates after the fire
post_burn_start_date = datetime.datetime(year=2015, month=8, day=1)
post_burn_end_date = datetime.datetime(year=2015, month=10, day=31)

# No Data Vaue
out_no_data_val = 0.0

In [ ]:
# Read the vector layer and make sure it is project using WGS84 (EPSG:4326)
vec_gdf = geopandas.read_file(vec_roi_file).to_crs(4326)

# Get layer bbox: minx, miny, maxx, maxy
gp_bbox = vec_gdf.total_bounds

# Create the GEE geometry from the bbox.
roi_west = gp_bbox[0]
roi_east = gp_bbox[2]
roi_north = gp_bbox[3]
roi_south = gp_bbox[1]
tile_aoi = ee.Geometry.BBox(roi_west, roi_south, roi_east, roi_north)

In [ ]:
# Get the landsat image collection
pre_burn_ls_img_col = pb_gee_tools.datasets.get_landsat_sr_collection(
    aoi=tile_aoi,
    start_date=pre_burn_start_date,
    end_date=pre_burn_end_date,
    cloud_thres=70,
    ignore_ls7=True,
    out_lstm_bands=False,
)

# Filter the collection to a specific row/path
pre_burn_ls_img_col = pre_burn_ls_img_col.filter(ee.Filter.eq("WRS_PATH", 203)).filter(
    ee.Filter.eq("WRS_ROW", 32)
)

# Create a median composite
pre_burn_ls_median = pre_burn_ls_img_col.median().unmask(out_no_data_val).clip(tile_aoi)

# Scene with the least cloud
pre_burn_ls_scn = pre_burn_ls_img_col.sort("CLOUD_COVER").first()

In [ ]:
pre_burn_ls_scn

In [ ]:
# Get the landsat image collection
post_burn_ls_img_col = pb_gee_tools.datasets.get_landsat_sr_collection(
    aoi=tile_aoi,
    start_date=post_burn_start_date,
    end_date=post_burn_end_date,
    cloud_thres=70,
    ignore_ls7=True,
    out_lstm_bands=False,
)

# Filter the collection to a specific row/path
post_burn_ls_img_col = post_burn_ls_img_col.filter(
    ee.Filter.eq("WRS_PATH", 203)
).filter(ee.Filter.eq("WRS_ROW", 32))

# Create a median composite
post_burn_ls_median = (
    post_burn_ls_img_col.median().unmask(out_no_data_val).clip(tile_aoi)
)

# Scene with least cloud
post_burn_ls_scn = post_burn_ls_img_col.sort("CLOUD_COVER").first()

In [ ]:
post_burn_ls_scn

In [ ]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

# Specify the visualisation options.
vis_params = {
    "min": [1320, 810, 20],
    "max": [4050, 4070, 1620],
    "bands": ["NIR", "SWIR1", "Red"],
}

# Add the Landsat Composite
map_obj.addLayer(pre_burn_ls_median, vis_params, "Pre-Burnt Landsat Median")
map_obj.addLayer(post_burn_ls_median, vis_params, "Post-Burnt Landsat Median")

# Add the Landsat Scenes
map_obj.addLayer(pre_burn_ls_scn, vis_params, "Pre-Burnt Landsat Scene")
map_obj.addLayer(post_burn_ls_scn, vis_params, "Post-Burnt Landsat Scene")

map_obj

In [ ]:
# Calculate the Normalised Burn Ratio (NBR) pre and post fire using the
# median composite
pre_med_nbr = pre_burn_ls_median.expression(
    "(NIR-SWIR)/(NIR+SWIR)",
    {
        "NIR": pre_burn_ls_median.select("NIR"),
        "SWIR": pre_burn_ls_median.select("SWIR1"),
    },
).rename("NBR")

post_med_nbr = post_burn_ls_median.expression(
    "(NIR-SWIR)/(NIR+SWIR)",
    {
        "NIR": post_burn_ls_median.select("NIR"),
        "SWIR": post_burn_ls_median.select("SWIR1"),
    },
).rename("NBR")


# Calculate the Normalised Burn Ratio (NBR) pre and post fire using the
# single selected scene
pre_scn_nbr = pre_burn_ls_scn.expression(
    "(NIR-SWIR)/(NIR+SWIR)",
    {"NIR": pre_burn_ls_scn.select("NIR"), "SWIR": pre_burn_ls_scn.select("SWIR1")},
).rename("NBR")

post_scn_nbr = post_burn_ls_scn.expression(
    "(NIR-SWIR)/(NIR+SWIR)",
    {"NIR": post_burn_ls_scn.select("NIR"), "SWIR": post_burn_ls_scn.select("SWIR1")},
).rename("NBR")

In [ ]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")

# Specify the visualisation options.
vis_params = {"min": [-0.2], "max": [0.2], "bands": ["NBR"]}

# Add the Landsat Composite
map_obj.addLayer(pre_med_nbr, vis_params, "Pre-Burnt Landsat Median NBR")
map_obj.addLayer(post_med_nbr, vis_params, "Post-Burnt Landsat Median NBR")

# Add the Landsat Scenes
map_obj.addLayer(pre_scn_nbr, vis_params, "Pre-Burnt Landsat Scene NBR")
map_obj.addLayer(post_scn_nbr, vis_params, "Post-Burnt Landsat Scene NBR")

map_obj

In [ ]:
# Create the Normalised Burn Ratio Difference (dNBR)
dNBR = pre_med_nbr.subtract(post_med_nbr).rename("dNBR")

In [ ]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")


# Specify the visualisation options.
vis_params = {
    "min": [1320, 810, 20],
    "max": [4050, 4070, 1620],
    "bands": ["NIR", "SWIR1", "Red"],
}

# Add the Landsat Composite
map_obj.addLayer(pre_burn_ls_median, vis_params, "Pre-Burnt Landsat Median")
map_obj.addLayer(post_burn_ls_median, vis_params, "Post-Burnt Landsat Median")

# Specify the visualisation options.
vis_dnbr_params = {"min": [-0.2], "max": [0.2], "bands": ["dNBR"]}

# Add the Landsat Composite
map_obj.addLayer(dNBR, vis_dnbr_params, "dNBR (Landsat Median)")

map_obj

In [ ]:
# Threshold the dNBR to identify the burnt area as a unique region
burnt_area = dNBR.gt(0.1).rename("burnt_area").toInt()
burnt_area = burnt_area.mask(burnt_area.eq(1))

In [ ]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")


# Specify the visualisation options.
vis_params = {
    "min": [1320, 810, 20],
    "max": [4050, 4070, 1620],
    "bands": ["NIR", "SWIR1", "Red"],
}

# Add the Landsat Composite
map_obj.addLayer(pre_burn_ls_median, vis_params, "Pre-Burnt Landsat Median")
map_obj.addLayer(post_burn_ls_median, vis_params, "Post-Burnt Landsat Median")

# Specify the visualisation options.
vis_dnbr_params = {"min": [-0.2], "max": [0.2], "bands": ["dNBR"]}

# Add the Landsat Composite
map_obj.addLayer(dNBR, vis_dnbr_params, "dNBR (Landsat Median)")

# Specify the visualisation options.
vis_burnt_params = {
    "min": 0,
    "max": 1,
    "palette": ["000000", "fcc200"],
    "bands": ["burnt_area"],
}

# Add the Landsat Composite
map_obj.addLayer(burnt_area, vis_burnt_params, "Burnt Area (Landsat Median)")

map_obj

In [ ]:
# Convert the burnt area to a vector layer
# (The burnt area region will be the largest polygon)
burnt_area_vec = burnt_area.reduceToVectors(
    geometry=tile_aoi,
    scale=30,
    geometryType="polygon",
    eightConnected=False,
    labelProperty="zone",
)

In [ ]:
burnt_area_vec.size()

In [ ]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")


# Specify the visualisation options.
vis_params = {
    "min": [1320, 810, 20],
    "max": [4050, 4070, 1620],
    "bands": ["NIR", "SWIR1", "Red"],
}

# Add the Landsat Composite
map_obj.addLayer(pre_burn_ls_median, vis_params, "Pre-Burnt Landsat Median")
map_obj.addLayer(post_burn_ls_median, vis_params, "Post-Burnt Landsat Median")

# Specify the visualisation options.
vis_dnbr_params = {"min": [-0.2], "max": [0.2], "bands": ["dNBR"]}

# Add the Landsat Composite
map_obj.addLayer(dNBR, vis_dnbr_params, "dNBR (Landsat Median)")


# Add the Landsat Composite
map_obj.addLayer(burnt_area_vec, {}, "Burnt Area (Vector)")

map_obj

In [ ]:
# Select the Largest Polygon
burnt_area_vec_area = burnt_area_vec.map(
    lambda feature: feature.set("area", feature.geometry().area(maxError=1))
)
burnt_area_vec_area = burnt_area_vec_area.sort("area", False)
largest_burnt_area_poly_vec = ee.FeatureCollection([burnt_area_vec_area.first()])

In [ ]:
# Check the size - there should just be 1 output polygon.
largest_burnt_area_poly_vec.size()

In [ ]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")


# Specify the visualisation options.
vis_params = {
    "min": [1320, 810, 20],
    "max": [4050, 4070, 1620],
    "bands": ["NIR", "SWIR1", "Red"],
}

# Add the Landsat Composite
map_obj.addLayer(pre_burn_ls_median, vis_params, "Pre-Burnt Landsat Median")
map_obj.addLayer(post_burn_ls_median, vis_params, "Post-Burnt Landsat Median")

# Specify the visualisation options.
vis_dnbr_params = {"min": [-0.2], "max": [0.2], "bands": ["dNBR"]}

# Add the Landsat Composite
map_obj.addLayer(dNBR, vis_dnbr_params, "dNBR (Landsat Median)")


# Add the Landsat Composite
map_obj.addLayer(largest_burnt_area_poly_vec, {}, "Burnt Area (Vector)")

map_obj

In [ ]:
# Rasterise the Largest Polygon.

# Specify the property (or properties) to use from the features.
properties = ["zone"]  # Replace 'zone' with the actual property name if different

# Specify the reducer to aggregate values (e.g., first, mean, median, etc.).
reducer = ee.Reducer.first()  # Use ee.Reducer.first() to take the first value

# Now call reduceToImage() with the required arguments.
burnt_area_msk_img = largest_burnt_area_poly_vec.reduceToImage(
    properties, reducer
).rename("burnt_area")

In [ ]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")


# Specify the visualisation options.
vis_params = {
    "min": [1320, 810, 20],
    "max": [4050, 4070, 1620],
    "bands": ["NIR", "SWIR1", "Red"],
}

# Add the Landsat Composite
map_obj.addLayer(pre_burn_ls_median, vis_params, "Pre-Burnt Landsat Median")
map_obj.addLayer(post_burn_ls_median, vis_params, "Post-Burnt Landsat Median")

# Specify the visualisation options.
vis_dnbr_params = {"min": [-0.2], "max": [0.2], "bands": ["dNBR"]}

# Add the Landsat Composite
map_obj.addLayer(dNBR, vis_dnbr_params, "dNBR (Landsat Median)")

# Specify the visualisation options.
vis_burnt_params = {"min": 0, "max": 1, "palette": ["000000", "fcc200"]}

# Add the Landsat Composite
map_obj.addLayer(burnt_area_msk_img, vis_burnt_params, "Burnt Area (Landsat Median)")

map_obj

In [ ]:
# Classify fire severity based on dNBR thresholds
fire_severity_img = (
    dNBR.expression(
        "dNBR < 0.1 ? 0 : "
        + "dNBR < 0.27 ? 1 : "
        + "dNBR < 0.44 ? 2 : "
        + "dNBR < 0.66 ? 3 : 4",
        {"dNBR": dNBR},
    )
    .rename("severity")
    .mask(burnt_area_msk_img.eq(1))
)

In [ ]:
map_obj = geemap.Map()

# Create the centre point for the map
cnt_pt_x = roi_west + (roi_east - roi_west) / 2
cnt_pt_y = roi_south + (roi_north - roi_south) / 2
cnt_pt = ee.Geometry.Point([cnt_pt_x, cnt_pt_y])
# Specify the centre point and scale of the map
map_obj.centerObject(cnt_pt, 12)

# Add Google Maps Basemap
gg_maps_url = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_maps_url, name="Google Maps", attribution="Google")

# Add Google Satellite Imagery Basemap
gg_img_url = "http://www.google.cn/maps/vt?lyrs=s@189&gl=cn&x={x}&y={y}&z={z}"
map_obj.add_tile_layer(gg_img_url, name="Google Imagery", attribution="Google")


# Specify the visualisation options.
vis_params = {
    "min": [1320, 810, 20],
    "max": [4050, 4070, 1620],
    "bands": ["NIR", "SWIR1", "Red"],
}

# Add the Landsat Composite
map_obj.addLayer(pre_burn_ls_median, vis_params, "Pre-Burnt Landsat Median")
map_obj.addLayer(post_burn_ls_median, vis_params, "Post-Burnt Landsat Median")

# Specify the visualisation options.
vis_dnbr_params = {"min": [-0.2], "max": [0.2], "bands": ["dNBR"]}

# Add the Landsat Composite
map_obj.addLayer(dNBR, vis_dnbr_params, "dNBR (Landsat Median)")

# Specify the visualisation options.
vis_burnt_params = {
    "min": 0,
    "max": 4,
    "palette": ["003f5c", "58508d", "bc5090", "ff6361", "ffa600"],
}

# Add the Landsat Composite
map_obj.addLayer(fire_severity_img, vis_burnt_params, "Burnt Severity")

map_obj

In [ ]:
# Create an image collection with the burnt area and fire severity images
out_img_collect = ee.ImageCollection([burnt_area_msk_img, fire_severity_img])
# Convert the image collection to a 2 band image which will be exported.
out_burn_img = out_img_collect.toBands().toInt().rename(["burnt_area", "severity"])

In [ ]:
out_burn_img

In [ ]:
# Output file name and directory
out_file_name = "fire_extent_severity"
out_gdrive_dir = "burnt_area_outputs"

# Run export to save image to google drive
task = ee.batch.Export.image.toDrive(
    image=out_burn_img,
    description=out_file_name,
    folder=out_gdrive_dir,
    scale=30,
    region=tile_aoi,
    fileFormat="GeoTIFF",
    formatOptions={"cloudOptimized": True, "noData": out_no_data_val},
)
task.start()

In [ ]:
# Run export to save image to google earth engine asset
# REMEMBER TO CREATE THE burnt_veg_example FOLDER!!
asset_id = f"projects/{ee_prj_name}/assets/burnt_veg_example/burnt_area_severity"
out_task_descrip = "fire_extent_severity_asset_export"

task = ee.batch.Export.image.toAsset(
    image=out_burn_img,
    description=out_task_descrip,
    assetId=asset_id,
    scale=30,
    region=tile_aoi,
    pyramidingPolicy={"burnt_area": "mode", "severity": "mode"},
)
task.start()